<a href="https://colab.research.google.com/github/GoncalSousa20/NAS_IPL/blob/main/EfficientNetV2S_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import os
# Define the paths to your three folders
train_path = '/content/drive/MyDrive/ProjetoCurso/train'
validation_path = '/content/drive/MyDrive/ProjetoCurso/validation'
test_path = '/content/drive/MyDrive/ProjetoCurso/test'

from keras.preprocessing.image import ImageDataGenerator
# Augment data
batch_size = 16
train_input_shape = (224, 224, 3)

train_datagen = ImageDataGenerator(rescale=1./255.,
                                   #rotation_range=45,
                                   #width_shift_range=0.5,
                                   #height_shift_range=0.5,
                                   shear_range=5,
                                   #zoom_range=0.7,
                                   horizontal_flip=True,
                                   vertical_flip=True,
                                  )

train_generator = train_datagen.flow_from_directory(directory='/content/drive/MyDrive/ProjetoCurso/train',
                                                    class_mode='categorical',
                                                    target_size=train_input_shape[0:2],
                                                    batch_size=batch_size,
                                                    shuffle=True,
                                                   )

valid_generator = train_datagen.flow_from_directory(directory='/content/drive/MyDrive/ProjetoCurso/validation',
                                                    class_mode='categorical',
                                                    target_size=train_input_shape[0:2],
                                                    batch_size=batch_size,
                                                    shuffle=True,
                                                   )

test_generator = train_datagen.flow_from_directory(directory='/content/drive/MyDrive/ProjetoCurso/test',
                                                    class_mode='categorical',
                                                    target_size=train_input_shape[0:2],
                                                    batch_size=1,
                                                    shuffle=False,
                                                   )

STEP_SIZE_TRAIN = train_generator.n//train_generator.batch_size
STEP_SIZE_VALID = valid_generator.n//valid_generator.batch_size
STEP_SIZE_TEST = test_generator.n//test_generator.batch_size
print("Total number of batches =", STEP_SIZE_TRAIN, "and", STEP_SIZE_VALID, "and", STEP_SIZE_TEST)

from tensorflow.keras.applications import EfficientNetV2S
base_model = EfficientNetV2S(weights='imagenet', include_top=False, input_shape=train_input_shape)
for layer in base_model.layers:
    layer.trainable = False

from keras.layers import Dense, Flatten, BatchNormalization, Activation, Dropout
from keras.models import Model

n_classes = 11

# Add layers at the end
X = base_model.output
X = Flatten()(X)

# First dense block
X = Dense(512, kernel_initializer='he_uniform')(X)
X = BatchNormalization()(X)
X = Activation('relu')(X)
# Optionally add dropout for regularization
# X = Dropout(0.5)(X)

# Second dense block
X = Dense(256, kernel_initializer='he_uniform')(X)
X = BatchNormalization()(X)
X = Activation('relu')(X)
# Optionally add dropout for regularization
# X = Dropout(0.5)(X)

# Third dense block
X = Dense(128, kernel_initializer='he_uniform')(X)
X = BatchNormalization()(X)
X = Activation('relu')(X)
# Optionally add dropout for regularization
# X = Dropout(0.5)(X)

# Fourth dense block
X = Dense(64, kernel_initializer='he_uniform')(X)
X = BatchNormalization()(X)
X = Activation('relu')(X)
# Optionally add dropout for regularization
# X = Dropout(0.5)(X)

# Fifth dense block
X = Dense(32, kernel_initializer='he_uniform')(X)
X = BatchNormalization()(X)
X = Activation('relu')(X)
# Optionally add dropout for regularization
# X = Dropout(0.5)(X)

# Final dense layer before output
X = Dense(16, kernel_initializer='he_uniform')(X)
X = BatchNormalization()(X)
X = Activation('relu')(X)
# Optionally add dropout for regularization
# X = Dropout(0.5)(X)

# Output layer
output = Dense(n_classes, activation='softmax')(X)

# Create the model
model = Model(inputs=base_model.input, outputs=output)

from keras.optimizers import Adam
optimizer = Adam(lr=0.0001)
model.compile(loss='categorical_crossentropy',
              optimizer=optimizer,
              metrics=['accuracy'])

from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score,
    matthews_corrcoef, cohen_kappa_score, hamming_loss, zero_one_loss, roc_auc_score,
    average_precision_score, brier_score_loss, classification_report
)
import pickle
import numpy as np
import tensorflow as tf
seed = 42
tf.random.set_seed(seed)
np.random.seed(seed)
# Additional custom metrics
def fbeta_score(precision, recall, beta=1.0):
    if precision + recall == 0:
        return 0.0
    return (1 + beta*2) * (precision * recall) / (beta*2 * precision + recall)

def g_mean(precision, recall):
    return np.sqrt(precision * recall)

# Example usage
dictionary_metrics = {
    'train_acc': [],
    'train_loss': [],
    'precision': [],
    'recall': [],
    'val_loss': [],
    'test_acc': [],
    'test_loss': [],
    'val_accuracy': [],
    'f1_score': [],
    'mcc': [],
    'cohen_kappa': [],
    'hamming_loss': [],
    'zero_one_loss': [],
    'auc_roc': [],
    'brier_score': [],
    'mAP': [],
    'f1_beta': [],
    'g_mean': []
}

# Function to get predictions and labels in batches
def get_predictions_and_labels(generator, model, steps):
    y_true = []
    y_pred = []
    y_pred_prob = []

    for i in range(steps):
        (X, y) = next(generator)
        y_pred_prob_batch = model.predict(X)
        y_pred_batch = np.argmax(y_pred_prob_batch, axis=1)

        y_pred.append(y_pred_batch)
        y_pred_prob.append(y_pred_prob_batch)
        y_true.append(y)

    # Flatten the lists
    y_pred = np.concatenate(y_pred, axis=0)
    y_pred_prob = np.concatenate(y_pred_prob, axis=0)
    y_true = np.concatenate(y_true, axis=0)

    # Update Truth vector based on argmax if y is one-hot encoded
    if y_true.ndim > 1 and y_true.shape[1] > 1:
        y_true = np.argmax(y_true, axis=1)

    # Convert to numpy arrays
    y_pred = np.asarray(y_pred).ravel()
    y_pred_prob = np.asarray(y_pred_prob)

    return y_true, y_pred,y_pred_prob

Mounted at /content/drive
Found 3862 images belonging to 11 classes.
Found 216 images belonging to 11 classes.
Found 219 images belonging to 11 classes.
Total number of batches = 241 and 13 and 219
82420632/82420632 [==============================] - 1s 0us/step


In [ ]:
for epoch in range(30):
  print(f"Starting Epoch {epoch+1}")
  history = model.fit(
      train_generator,
      validation_data=valid_generator,
      epochs=1
  )
  print(history.history)
  print(f"Finished Epoch {epoch+1}")

  # Extract validation labels
  val_labels = []
  for i in range(len(valid_generator)):
      _, batch_labels = next(valid_generator)
      val_labels.extend(batch_labels)

  val_labels = np.array(val_labels)
  dictionary_metrics['train_acc'].append(history.history["accuracy"])
  dictionary_metrics['train_loss'].append(history.history["loss"])

  # Calculate metrics for validation and test sets
  val_loss, val_accuracy = model.evaluate(valid_generator)
  dictionary_metrics['val_accuracy'].append(val_accuracy)
  dictionary_metrics['val_loss'].append(val_loss)
  test_loss, test_acc = model.evaluate(test_generator)
  dictionary_metrics['test_loss'].append(test_loss)
  dictionary_metrics['test_acc'].append(test_acc)

  test_labels, test_pred_labels, test_pred_prob = get_predictions_and_labels(test_generator, model, steps=STEP_SIZE_TEST)

  num_classes = len(np.unique(test_labels))

  test_classification_report = classification_report(test_labels, test_pred_labels, output_dict=True)
  test_recall_list = []
  test_precision_list = []
  test_f1_score_list = []
  test_f1_beta_list = []
  test_g_mean_list = []

  for class_label in range(num_classes):
      recall = test_classification_report[str(class_label)]['recall']
      precision = test_classification_report[str(class_label)]['precision']
      f1 = test_classification_report[str(class_label)]['f1-score']
      test_recall_list.append(recall)
      test_precision_list.append(precision)
      test_f1_score_list.append(f1)
      test_f1_beta_list.append(fbeta_score(precision, recall, beta=2.0))  # Using beta=2 for F1-beta score
      test_g_mean_list.append(g_mean(precision, recall))

  dictionary_metrics['recall'].append(test_recall_list)
  dictionary_metrics['precision'].append(test_precision_list)
  dictionary_metrics['f1_score'].append(test_f1_score_list)
  dictionary_metrics['f1_beta'].append(test_f1_beta_list)
  dictionary_metrics['g_mean'].append(test_g_mean_list)
  dictionary_metrics['mcc'].append(matthews_corrcoef(test_labels, test_pred_labels))
  dictionary_metrics['cohen_kappa'].append(cohen_kappa_score(test_labels, test_pred_labels))
  dictionary_metrics['hamming_loss'].append(hamming_loss(test_labels, test_pred_labels))
  dictionary_metrics['zero_one_loss'].append(zero_one_loss(test_labels, test_pred_labels, normalize=True))

  try:
      test_roc_auc = roc_auc_score(test_labels, test_pred_prob, multi_class='ovr')
      dictionary_metrics['auc_roc'].append(test_roc_auc)
  except ValueError:
      print("Error in calculating ROC AUC for test set")

  try:
      avg_precision = []
      for i in range(num_classes):
          avg_precision.append(average_precision_score((test_labels == i).astype(int), test_pred_prob[:, i]))
      avg_precision = np.mean(avg_precision)
      dictionary_metrics['mAP'].append(avg_precision)
  except ValueError as e:
      print(f"Error in calculating Average Precision: {e}")

  # Calculate Brier score for each class and average them
  brier_scores = []
  for i in range(num_classes):
      true_binary = (test_labels == i).astype(float)
      probas = test_pred_prob[:, i]
      brier_score = brier_score_loss(true_binary, probas)
      brier_scores.append(brier_score)

  average_brier_score = np.mean(brier_scores)
  dictionary_metrics['brier_score'].append(average_brier_score)

with open('/content/drive/MyDrive/ProjetoCurso/metrics_EfficientNetVS.pkl', 'wb') as fp:
    pickle.dump(dictionary_metrics,fp)

Starting Epoch 1
242/242 [==============================] - 1834s 7s/step - loss: 2.3362 - accuracy: 0.1844 - val_loss: 2.4194 - val_accuracy: 0.1620
{'loss': [2.336195468902588], 'accuracy': [0.18436042964458466], 'val_loss': [2.4194116592407227], 'val_accuracy': [0.16203702986240387]}
Finished Epoch 1
1/1 [==============================] - 0s 32ms/step
Starting Epoch 2


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


242/242 [==============================] - 95s 394ms/step - loss: 2.1802 - accuracy: 0.2434 - val_loss: 2.1059 - val_accuracy: 0.2824
{'loss': [2.180243968963623], 'accuracy': [0.24339720606803894], 'val_loss': [2.105928421020508], 'val_accuracy': [0.28240740299224854]}
Finished Epoch 2
1/1 [==============================] - 0s 31ms/step
Starting Epoch 3


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


242/242 [==============================] - 95s 392ms/step - loss: 2.1337 - accuracy: 0.2613 - val_loss: 2.1244 - val_accuracy: 0.2454
{'loss': [2.133718967437744], 'accuracy': [0.26126357913017273], 'val_loss': [2.1243667602539062], 'val_accuracy': [0.24537037312984467]}
Finished Epoch 3
1/1 [==============================] - 0s 32ms/step
Starting Epoch 4


...

242/242 [==============================] - 97s 402ms/step - loss: 1.8051 - accuracy: 0.3610 - val_loss: 2.2339 - val_accuracy: 0.2546
{'loss': [1.8050988912582397], 'accuracy': [0.36095288395881653], 'val_loss': [2.233881950378418], 'val_accuracy': [0.25462964177131653]}
Finished Epoch 25
1/1 [==============================] - 0s 37ms/step
Starting Epoch 26


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


242/242 [==============================] - 98s 404ms/step - loss: 1.8054 - accuracy: 0.3571 - val_loss: 1.9742 - val_accuracy: 0.2778
{'loss': [1.8054025173187256], 'accuracy': [0.35706886649131775], 'val_loss': [1.9741783142089844], 'val_accuracy': [0.2777777910232544]}
Finished Epoch 26
1/1 [==============================] - 0s 38ms/step
Starting Epoch 27
242/242 [==============================] - 101s 417ms/step - loss: 1.8193 - accuracy: 0.3604 - val_loss: 2.3850 - val_accuracy: 0.2454
{'loss': [1.8192559480667114], 'accuracy': [0.36043500900268555], 'val_loss': [2.385026454925537], 'val_accuracy': [0.24537037312984467]}
Finished Epoch 27
1/1 [==============================] - 0s 33ms/step
Starting Epoch 28


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


242/242 [==============================] - 98s 404ms/step - loss: 1.8073 - accuracy: 0.3506 - val_loss: 1.9860 - val_accuracy: 0.2778
{'loss': [1.8073170185089111], 'accuracy': [0.35059553384780884], 'val_loss': [1.985969066619873], 'val_accuracy': [0.2777777910232544]}
Finished Epoch 28
1/1 [==============================] - 0s 33ms/step
Starting Epoch 29
242/242 [==============================] - 97s 402ms/step - loss: 1.8149 - accuracy: 0.3607 - val_loss: 1.9206 - val_accuracy: 0.3194
{'loss': [1.8149311542510986], 'accuracy': [0.36069393157958984], 'val_loss': [1.9206483364105225], 'val_accuracy': [0.3194444477558136]}
Finished Epoch 29
1/1 [==============================] - 0s 32ms/step
Starting Epoch 30
242/242 [==============================] - 98s 405ms/step - loss: 1.7929 - accuracy: 0.3672 - val_loss: 2.0215 - val_accuracy: 0.2685
{'loss': [1.792938232421875], 'accuracy': [0.36716726422309875], 'val_loss': [2.0215017795562744], 'val_accuracy': [0.26851850748062134]}
Finished 

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pickle

# Function to print the metrics in an organized way
def print_metrics(metrics):
    print("Metrics:")
    print("--------")
    for key, value in metrics.items():
        if isinstance(value, list):  # Check if the value is a list (e.g., confusion matrix)
            print(f"{key}:")
            for row in value:
                print(f"  {row}")
        else:
            print(f"{key}: {value}")

# Reading the metrics from the pickle file
try:
    with open('/content/drive/MyDrive/ProjetoCurso/metrics_EfficientNetV2S.pkl', 'rb') as fp:
        loaded_metrics = pickle.load(fp)
    print("Metrics successfully loaded from metrics.pkl")
    print_metrics(loaded_metrics)
except EOFError:
    print("Error: Ran out of input - the file may be empty or improperly written.")
except FileNotFoundError:
    print("Error: The file does not exist.")
except pickle.UnpicklingError:
    print("Error: The file contents are not a valid pickle.")


Metrics successfully loaded from metrics.pkl
Metrics:
--------
train_acc:
  [0.18436042964458466]
  [0.24339720606803894]
  [0.26126357913017273]
  [0.27446919679641724]
  [0.2786121070384979]
  [0.2975142300128937]
  [0.29129984974861145]
  [0.30165717005729675]
  [0.2962195873260498]
  [0.30787158012390137]
  [0.3140859603881836]
  [0.3239254355430603]
  [0.3169342279434204]
  [0.3303987681865692]
  [0.3353184759616852]
  [0.33298808336257935]
  [0.3368721008300781]
  [0.3247022330760956]
  [0.33635422587394714]
  [0.34360435605049133]
  [0.33946141600608826]
  [0.3415328860282898]
  [0.35370275378227234]
  [0.35344380140304565]
  [0.36095288395881653]
  [0.35706886649131775]
  [0.36043500900268555]
  [0.35059553384780884]
  [0.36069393157958984]
  [0.36716726422309875]
train_loss:
  [2.336195468902588]
  [2.180243968963623]
  [2.133718967437744]
  [2.088470458984375]
  [2.066725969314575]
  [2.0328874588012695]
  [2.0093436241149902]
  [1.9841430187225342]
  [1.9845407009124756]
  [

## Training graph

In [ ]:
# Merge history1 and history2
history = {}
history['loss'] = history1.history['loss'] + history2.history['loss']
history['acc'] = history1.history['acc'] + history2.history['acc']
history['val_loss'] = history1.history['val_loss'] + history2.history['val_loss']
history['val_acc'] = history1.history['val_acc'] + history2.history['val_acc']
history['lr'] = history1.history['lr'] + history2.history['lr']

In [ ]:
# Plot the training graph
def plot_training(history):
    acc = history['acc']
    val_acc = history['val_acc']
    loss = history['loss']
    val_loss = history['val_loss']
    epochs = range(len(acc))

    fig, axes = plt.subplots(1, 2, figsize=(15,5))

    axes[0].plot(epochs, acc, 'r-', label='Training Accuracy')
    axes[0].plot(epochs, val_acc, 'b--', label='Validation Accuracy')
    axes[0].set_title('Training and Validation Accuracy')
    axes[0].legend(loc='best')

    axes[1].plot(epochs, loss, 'r-', label='Training Loss')
    axes[1].plot(epochs, val_loss, 'b--', label='Validation Loss')
    axes[1].set_title('Training and Validation Loss')
    axes[1].legend(loc='best')

    plt.show()

plot_training(history)

## Evaluate performance

In [ ]:
# Prediction accuracy on train data
score = model.evaluate_generator(train_generator, verbose=1)
print("Prediction accuracy on train data =", score[1])

In [ ]:
# Prediction accuracy on CV data
score = model.evaluate_generator(valid_generator, verbose=1)
print("Prediction accuracy on CV data =", score[1])

In [ ]:
# Classification report and confusion matrix
from sklearn.metrics import *
import seaborn as sns

tick_labels = artists_top_name.tolist()

def showClassficationReport_Generator(model, valid_generator, STEP_SIZE_VALID):
    # Loop on each generator batch and predict
    y_pred, y_true = [], []
    for i in range(STEP_SIZE_VALID):
        (X,y) = next(valid_generator)
        y_pred.append(model.predict(X))
        y_true.append(y)

    # Create a flat list for y_true and y_pred
    y_pred = [subresult for result in y_pred for subresult in result]
    y_true = [subresult for result in y_true for subresult in result]

    # Update Truth vector based on argmax
    y_true = np.argmax(y_true, axis=1)
    y_true = np.asarray(y_true).ravel()

    # Update Prediction vector based on argmax
    y_pred = np.argmax(y_pred, axis=1)
    y_pred = np.asarray(y_pred).ravel()

    # Confusion Matrix
    fig, ax = plt.subplots(figsize=(10,10))
    conf_matrix = confusion_matrix(y_true, y_pred, labels=np.arange(n_classes))
    conf_matrix = conf_matrix/np.sum(conf_matrix, axis=1)
    sns.heatmap(conf_matrix, annot=True, fmt=".2f", square=True, cbar=False,
                cmap=plt.cm.jet, xticklabels=tick_labels, yticklabels=tick_labels,
                ax=ax)
    ax.set_ylabel('Actual')
    ax.set_xlabel('Predicted')
    ax.set_title('Confusion Matrix')
    plt.show()

    print('Classification Report:')
    print(classification_report(y_true, y_pred, labels=np.arange(n_classes), target_names=artists_top_name.tolist()))

showClassficationReport_Generator(model, valid_generator, STEP_SIZE_VALID)

# Evaluate performance by predicting on random images from dataset

In [ ]:
# Prediction
from keras.preprocessing import *

n = 5
fig, axes = plt.subplots(1, n, figsize=(25,10))

for i in range(n):
    random_artist = random.choice(artists_top_name)
    random_image = random.choice(os.listdir(os.path.join(images_dir, random_artist)))
    random_image_file = os.path.join(images_dir, random_artist, random_image)

    # Original image

    test_image = image.load_img(random_image_file, target_size=(train_input_shape[0:2]))

    # Predict artist
    test_image = image.img_to_array(test_image)
    test_image /= 255.
    test_image = np.expand_dims(test_image, axis=0)

    prediction = model.predict(test_image)
    prediction_probability = np.amax(prediction)
    prediction_idx = np.argmax(prediction)

    labels = train_generator.class_indices
    labels = dict((v,k) for k,v in labels.items())

    #print("Actual artist =", random_artist.replace('_', ' '))
    #print("Predicted artist =", labels[prediction_idx].replace('_', ' '))
    #print("Prediction probability =", prediction_probability*100, "%")

    title = "Actual artist = {}\nPredicted artist = {}\nPrediction probability = {:.2f} %" \
                .format(random_artist.replace('_', ' '), labels[prediction_idx].replace('_', ' '),
                        prediction_probability*100)

    # Print image
    axes[i].imshow(plt.imread(random_image_file))
    axes[i].set_title(title)
    axes[i].axis('off')

plt.show()